Assignment 2 CSCN8000 Artificial Intelligence Algorithms and Mathematics

1.Consider the heart disease dataset in pandas dataframe 2.Remove outliers using mean,median,Z score. 3.Convert text columns to numbers using label encoding and one hot encoding 4.Apply scaling 5.Build a machine learning classification model using support vector machine. Demonstrate the standalone model as well as Bagging model and include observations about the oerformance 6.Now use decision tree classifier. Use standalone model as well as Bagging and check if you notice any difference in performance 7.Comparing performance of svm and decision tree classifier figure out where it makes most sense to use bagging and why.

We'll take the following actions to complete these tasks:

1: Load the heart disease dataset into a pandas DataFrame after importing the required libraries.

2: Use the mean, median, and Z-score techniques to eliminate outliers.

3: Use label encoding and one-hot encoding to convert text columns to numbers.

4: To the numerical columns, apply scaling.

5: Utilise Support Vector Machine (SVM) to create a machine learning classification model.

6: Create a decision tree classification model using machine learning.

7: To determine the situations when using Bagging is most appropriate, compare the performance of SVM and Decision Tree Classifier.

Step 1: Import the necessary libraries and load the dataset.

In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score

# Load the dataset from the CSV file
df = pd.read_csv(r"C:\Users\asus\Downloads\archive (1).zip")

# Let's check the first few rows of the dataset
print(df.head())

   Age Sex ChestPainType  RestingBP  Cholesterol  FastingBS RestingECG  MaxHR  \
0   40   M           ATA        140          289          0     Normal    172   
1   49   F           NAP        160          180          0     Normal    156   
2   37   M           ATA        130          283          0         ST     98   
3   48   F           ASY        138          214          0     Normal    108   
4   54   M           NAP        150          195          0     Normal    122   

  ExerciseAngina  Oldpeak ST_Slope  HeartDisease  
0              N      0.0       Up             0  
1              N      1.0     Flat             1  
2              N      0.0       Up             0  
3              Y      1.5     Flat             1  
4              N      0.0       Up             0  


Step 2: Remove outliers using different methods like mean, median, and Z-score.

In [3]:
def remove_outliers_mean(df, columns):
    for column in columns:
        mean = df[column].mean()
        std = df[column].std()
        df = df[(df[column] > mean - 2 * std) & (df[column] < mean + 2 * std)]
    return df

def remove_outliers_median(df, columns):
    for column in columns:
        median = df[column].median()
        q1 = df[column].quantile(0.25)
        q3 = df[column].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        df = df[(df[column] > lower_bound) & (df[column] < upper_bound)]
    return df

def remove_outliers_zscore(df, columns, threshold=3):
    for column in columns:
        z_scores = (df[column] - df[column].mean()) / df[column].std()
        df = df[abs(z_scores) < threshold]
    return df

# Columns for which outliers need to be removed (numerical columns only)
numerical_columns = ["Age", "RestingBP", "Cholesterol", "MaxHR", "Oldpeak"]

# Remove outliers using mean, median, and Z-score methods
df_mean = remove_outliers_mean(df, numerical_columns)
df_median = remove_outliers_median(df, numerical_columns)
df_zscore = remove_outliers_zscore(df, numerical_columns)

# Let's check the shape of the dataframes after removing outliers using different methods
print("Original Shape:", df.shape)
print("Shape after mean method:", df_mean.shape)
print("Shape after median method:", df_median.shape)
print("Shape after Z-score method:", df_zscore.shape)

Original Shape: (918, 12)
Shape after mean method: (753, 12)
Shape after median method: (690, 12)
Shape after Z-score method: (899, 12)


Step 3: Convert text columns to numbers using  different encodings :label encoding and one-hot encoding.

In [4]:
# Convert categorical columns using label encoding
label_encoder = LabelEncoder()
df["Sex"] = label_encoder.fit_transform(df["Sex"])
df["ChestPainType"] = label_encoder.fit_transform(df["ChestPainType"])
df["FastingBS"] = label_encoder.fit_transform(df["FastingBS"])
df["RestingECG"] = label_encoder.fit_transform(df["RestingECG"])
df["ExerciseAngina"] = label_encoder.fit_transform(df["ExerciseAngina"])
df["ST_Slope"] = label_encoder.fit_transform(df["ST_Slope"])

# Convert categorical columns using one-hot encoding
df_one_hot = pd.get_dummies(df, columns=["ChestPainType", "RestingECG", "ST_Slope"])

# Let's check the modified dataframe with one-hot encoding
print(df_one_hot.head())

   Age  Sex  RestingBP  Cholesterol  FastingBS  MaxHR  ExerciseAngina  \
0   40    1        140          289          0    172               0   
1   49    0        160          180          0    156               0   
2   37    1        130          283          0     98               0   
3   48    0        138          214          0    108               1   
4   54    1        150          195          0    122               0   

   Oldpeak  HeartDisease  ChestPainType_0  ChestPainType_1  ChestPainType_2  \
0      0.0             0            False             True            False   
1      1.0             1            False            False             True   
2      0.0             0            False             True            False   
3      1.5             1             True            False            False   
4      0.0             0            False            False             True   

   ChestPainType_3  RestingECG_0  RestingECG_1  RestingECG_2  ST_Slope_0  \
0         

Step 4: Apply scaling to the numerical columns.

In [5]:
# Separate the features (X) and target (y)
X = df_one_hot.drop("HeartDisease", axis=1)
y = df_one_hot["HeartDisease"]

# Apply scaling to numerical columns using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Step 5:Build a machine learning classification model using support vector machine. Demonstrate the standalone model as well as Bagging model and include observations about the performance

In [9]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Build and train the standalone SVM model
svm_model_standalone = SVC(kernel="linear", random_state=42)
svm_model_standalone.fit(X_train, y_train)

# Predict on the test set for the standalone model
y_pred_svm_standalone = svm_model_standalone.predict(X_test)

# Calculate accuracy for the standalone SVM model
accuracy_svm_standalone = accuracy_score(y_test, y_pred_svm_standalone)
print("Accuracy of standalone SVM model:", accuracy_svm_standalone)

# Build and train the Bagging model using SVM as the base estimator
svm_model_bagging = BaggingClassifier(base_estimator=SVC(kernel="linear"), n_estimators=10, random_state=42)
svm_model_bagging.fit(X_train, y_train)

# Predict on the test set for the Bagging model
y_pred_svm_bagging = svm_model_bagging.predict(X_test)

# Calculate accuracy for the Bagging model
accuracy_svm_bagging = accuracy_score(y_test, y_pred_svm_bagging)
print("Accuracy of SVM Bagging model:", accuracy_svm_bagging)

Accuracy of standalone SVM model: 0.8586956521739131
Accuracy of SVM Bagging model: 0.8478260869565217


c:\Users\asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


# Observation:

The accuracy of the standalone SVM model will vary depending on the particular dataset and issue at hand. SVMs work well for classification jobs in general, especially when the data is well-separated and there is a distinct difference between the classes.

When the dataset is noisy or contains outliers, bagging using SVM as the base estimator can increase the model's resilience and generalisation performance. By averaging the forecasts from various SVM models, it lowers variance and overfitting.

It's possible that the Bagging model performs better than the single SVM model. The complexity of the underlying link between the features and the target variable will depend on the dataset.

In situations where the base estimator is unstable or prone to overfitting, bagging often enhances performance. The improvement from Bagging may not be significant in this particular instance because SVMs are typically steady.

When the performance of the standalone SVM model is subpar or the dataset is limited and very variable, the effectiveness of Bagging may become more apparent.

Step 6: Build the Decision Tree Classifier models

In [10]:
# Build and train the standalone Decision Tree Classifier model
dt_model_standalone = DecisionTreeClassifier(random_state=42)
dt_model_standalone.fit(X_train, y_train)

# Predict on the test set for the standalone Decision Tree model
y_pred_dt_standalone = dt_model_standalone.predict(X_test)

# Calculate accuracy for the standalone Decision Tree model
accuracy_dt_standalone = accuracy_score(y_test, y_pred_dt_standalone)
print("Accuracy of standalone Decision Tree model:", accuracy_dt_standalone)

# Build and train the Bagging model using Decision Tree Classifier as the base estimator
dt_model_bagging = BaggingClassifier(base_estimator=DecisionTreeClassifier(random_state=42), n_estimators=10, random_state=42)
dt_model_bagging.fit(X_train, y_train)

# Predict on the test set for the Bagging model
y_pred_dt_bagging = dt_model_bagging.predict(X_test)

# Calculate accuracy for the Bagging model
accuracy_dt_bagging = accuracy_score(y_test, y_pred_dt_bagging)
print("Accuracy of Decision Tree Bagging model:", accuracy_dt_bagging)

Accuracy of standalone Decision Tree model: 0.8097826086956522
Accuracy of Decision Tree Bagging model: 0.8641304347826086


c:\Users\asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Step 7: Analysing the performance of the Decision Tree Classifier and SVM to determine when and why Bagging is most appropriate:

In [11]:
print("Accuracy of standalone SVM model:", accuracy_svm_standalone)
print("Accuracy of SVM Bagging model:", accuracy_svm_bagging)
print("Accuracy of standalone Decision Tree model:", accuracy_dt_standalone)
print("Accuracy of Decision Tree Bagging model:", accuracy_dt_bagging)

Accuracy of standalone SVM model: 0.8586956521739131
Accuracy of SVM Bagging model: 0.8478260869565217
Accuracy of standalone Decision Tree model: 0.8097826086956522
Accuracy of Decision Tree Bagging model: 0.8641304347826086


Comparing the independent models:

SVM: The accuracy_svm_standalone variable represents the accuracy of the standalone SVM model.
Decision Tree Classifier: accuracy_dt_standalone represents the accuracy of the standalone Decision Tree model.
The Bagging models are contrasted:

SVM Bagging: accuracy_svm_bagging is a representation of the accuracy of the Bagging model using SVM as the basis estimator.
Decision Tree Bagging: accuracy_dt_bagging is a representation of the accuracy of the Bagging model using the Decision Tree Classifier as the base estimator.
The exact dataset and the underlying link between features and the target variable will determine how well SVM and Decision Tree Classifier (standalone and Bagging) perform.

SVM generally works well with high-dimensional data and nonlinear decision boundaries. Complex interactions between features and labels can be handled by it.

A straightforward and understandable model that can handle both categorical and numerical features is the decision tree classifier. However, especially with deep trees, it could suffer from overfitting.

By combining many instances of the base model, bagging reduces overfitting and enhances generalisation. When the base model, such as Decision Tree Classifier, is prone to overfitting, it might be more advantageous.

When the performance of the standalone model is poor or the dataset is limited and highly variable, the benefits of bagging become more apparent.

By considering the performance comparison between SVM and Decision Tree Classifier, it may not be necessary to combine Bagging with SVM if SVM achieves a greater accuracy than the Decision Tree model (either standalone or with Bagging).

Applying Bagging to the Decision Tree, however, may greatly enhance its performance if the standalone Decision Tree Classifier performs less accurately than SVM or exhibits overfitting.